# CMIP6 Arctic Case Study : Compute the aerosol-cloud interaction effective radiative forcing

--- 

## What this notebook holds :

---

<span style="color:red">**It is recommended to first run the *get_cmip6_data.ipynb* notebook to download and prepare the raw model's outputs data for the analysis.**</span>

This notebook aims at showing the effective radiative forcing that is caused by the aerosol burdens' levels of 2014 in the shortwabe. It aims at evaluating the different components of the aerosol-cloud's interaction impact on the Arctic warming. This is done using a subset of the CMIP6 model ensemble and using the APRP method devised by *Taylor and al. (2007)* and whose implementation was done by *Zelinka and al. (2023)*. 

The subset used is described in the github repository and is the one used by *Zelinka and al.*. It can be found in details in the *load_cmip6* submodule as the case described by *ZELINKA-SW* in the *set_search_criterias* function. 

### The use of CMIP6 data

We use two experiments realized during the CMIP6  : **piClim-control** and **piClim-aer**. These are both atmosphere-only climate model simulations in which sea surface temperatures (SSTs) and sea icea concentrations (SICs) are fixed at model-specific preindustrial climatological values. The description of the experiments can be found here : https://wcrp-cmip.github.io/CMIP6_CVs/docs/CMIP6_experiment_id.html. On the one hand, the **piClim-control** realization assumes aerosols burdens set to their preindustrial levels hence why it is dubbed as the control experiment. On the other hand, the **piClim-aer** realization uses present-day, present-day being 2014, aerosols burdens' levels.

The variable used are listed and explicited below according to : https://clipc-services.ceda.ac.uk/dreq/mipVars.html. All the variables are monthly timeseries over 30 years. We use the monthly climatology of each of these variables.

> <span style="color:SkyBlue">**clt**</span>  : Total cloud area fraction (%) for the whole atmospheric column
>
> <span style="color:gold">**rsdt**</span> : Shortwave radiation ($W/m^{2}$) **incident** at the TOA
> 
> <span style="color:orange">**rsut**</span> : Shortwave radiation ($W/m^{2}$) **going out**  at the TOA
>
> <span style="color:orangered">**rsutcs**</span> : Shortwave radiation ($W/m^{2}$) **going out**  at TOA for **clear-sky conditions**
> 
> <span style="color:Orchid">**rsds**</span> : Shortwave **downwelling** radiation ($W/m^{2}$) at the surface
> 
> <span style="color:Indigo ">**rsdscs**</span>  : Shortwave **downwelling** radiation ($W/m^{2}$) at the surface for **clear-sky conditions**
> 
> <span style="color:YellowGreen">**rsus**</span> : Shortwave **upwelling** radiation ($W/m^{2}$) at the surface
>
> <span style="color:Darkgreen">**rsuscs**</span>: Shortwave **upwelling** radiation ($W/m^{2}$) at the surface for **clear-sky conditions**
>
> **areacella** : For every grid, the latitude-dependent surface associated to each grid point.

### Approximate Partial Radiative Perturbation

**TO WRITE...**

## References : 

### APRP method

Taylor, K. E., M. Crucifix, P. Braconnot, C. D. Hewitt, C. Doutriaux, A. J. Broccoli, J. F. B. Mitchell, and M. J. Webb, 2007: Estimating Shortwave Radiative Forcing and Response in Climate Models. J. Climate, 20, 2530–2543, https://doi.org/10.1175/JCLI4143.1. 

### APRP module used for our analysis

https://github.com/mzelinka/aprp

Zelinka, M. D., Smith, C. J., Qin, Y., and Taylor, K. E.: Comparison of methods to estimate aerosol effective radiative forcings in climate models, Atmos. Chem. Phys., 23, 8879–8898, https://doi.org/10.5194/acp-23-8879-2023, 2023.

### Author

Giboni Lucas, 1st year PhD Candidate at IGE (CNRS), Grenoble.

https://github.com/gibonil/gibonil

### License

Feel free to share, use and improve the following code according to the provided license on the repository.

---

# Initialisation

---

## Importations

We import the needed libraries

In [ ]:
# ================ IMPORTATIONS ================ #

### LOAD AND NAVIGATE THROUGH THE DATA ###

import os  # to get access to commands related to path setting and creation of directories

### DATA OBJECTS AND ASSOCIATED COMPUTATION ###

import numpy as np  # to handle numpy arrays and the associated tools

import xarray as xr  # to manage the data

import xcdat as xc  # to handle climate model outputs with xarray

### REPRESENTING DATA ###

import matplotlib.pyplot as plt  # to handle plotting routines

import cartopy.crs as ccrs  # to handle map projections

### HOMEMADE LIBRARIES ###

## Load the climatology dictionary ##

from utilities.get_cmip6_data.store_data.dict_netcdf_transform import (
    netcdf_to_dict, # to load the climatology dictionary
)  

### APRP LIBRARY ###

from utilities.aprp_library.code.aprp import(
    APRP, # APRP method
)

### Define the paths for saving and loading data

In this part, the user needs to define the paths at which will be downloaded the data and saved the climatologies if necessary. They also need to define the path to the table associating the monthly climatologies netcdf files with their respective key in the climatologies' dictionary to be loaded.

If *get_cmip6_data.ipynb* notebook was run before, then only the path to the *key_paths_table.pkl* file is relevant here.

In [ ]:
# ================ DEFINE THE FOLDERS WHERE IS STORED THE DATA ================ #

### DEFINE THE HOME DIRECTORY ###

## Home directory ##

homedir_path = os.path.expanduser("~")

### DEFINE WHERE TO DOWNLOAD THE RAW DATA ###

## Parent directory of the download folder ##

parent_path_download = homedir_path + "/certainty-data"

## Name of the download folder ##

download_folder_name = "CMIP6-DATA"

### DEFINE WHERE TO SAVE THE CLIMATOLOGIES ###

## Path of the save directory ##

parent_path_save_clim = (
    homedir_path + "/certainty-data/" + download_folder_name + "/climatologies"
)

### DEFINE WHERE TO LOOK FOR THE TABLE OF THE CLIMATOLOGIES' PATHS ###

table_path = parent_path_save_clim + "/table" + "/key_paths_table.pkl"

## Load the CMIP6 climatology

We load the CMIP6 climatology. We first check the existence of the *key_paths_table.pkl* file that indicate that the *get_cmip6_data.ipynb* notebook was successfully run. Otherwise, we launch the full routine to download the raw data and generate the climatologies. These paths are the absolute paths from the home directory.

**This full routine lasts about 2 hours if no download was done before.**

In [ ]:
# ================ LOAD THE MONTHLY CLIMATOLOGIES ================ #

### CHECK IF THE CLIMATOLOGIES WERE GENERATED BEFORE ###

if os.path.lexists(path = table_path) :

    ## The table exists we can therefore load the dictionary of the climatologies ## 

    print("The monthly climatologies were generated and can be retrieved")

    data_cmip6_clim = netcdf_to_dict(save_path=parent_path_save_clim)

else :

    ## The table does not exist : we need to download the data and prepare it ##

    print("No key_paths_table.pkl at the given path.\n" \
    "We download the data and create the monthly climatologies dictionary")

    # We import the necessary submodule #

    from utilities.get_cmip6_data.prepare_data.extract_climatologies import (
    create_climatology_dict,  # to create the climatology dictionary and save it
    )

    ## Create the climatologies dictionary ##

    create_climatology_dict(
    data_path = parent_path_download,
    data_folder_name = download_folder_name,
    save_path = parent_path_save_clim,
    selected_case = "ZELINKA-SW",
)
    
    ## Load it ##

    data_cmip6_clim = netcdf_to_dict(save_path=parent_path_save_clim)

The monthly climatologies were generated and can be retrieved


## APRP METHOD

In [6]:
clim_keys = list(data_cmip6_clim.keys())

clim_keys_without_exp = [
    get_model_entries__only_from_clim(key_with_exp) for key_with_exp in clim_keys
]


# np.unique(clim_keys_without_exp) + ".piClim-control"

# np.unique(clim_keys_without_exp) + ".piClim-aer"

In [7]:
dict_aprp = {}

for key in clim_keys_without_exp:

    key_control = key + ".piClim-control"

    key_aer = key + ".piClim-aer"

    output = APRP(data_cmip6_clim[key_control], data_cmip6_clim[key_aer])

    dict_aprp[key] = output

# add units to lat when building the dataset

In [8]:
dict_aprp[clim_keys_without_exp[0]]

when to regrid ????

## REGRIDDING

create output grid

In [9]:
import numpy.typing as npt


def get_step(coordinate_array: npt.NDArray[np.float64]) -> float:
    """ """

    return np.max(np.diff(coordinate_array))

In [10]:
list_steps_lat = [get_step(dict_aprp[key].lat.values) for key in clim_keys_without_exp]

list_steps_lon = [get_step(dict_aprp[key].lon.values) for key in clim_keys_without_exp]

In [ ]:
max_step_lon = np.max(list_steps_lon)

max_step_lat = np.max(list_steps_lat)

to have a regular grid 

In [21]:
from math import floor

n_steps_lon = floor(360 / max_step_lon)  # bc we want a step greater than max_step_lat

new_step_lon = 360 / n_steps_lon

In [22]:
n_steps_lat = floor(180 / max_step_lat)  # bc we want a step greater than max_step_lat

new_step_lat = 180 / n_steps_lat

In [23]:
(90 - new_step_lat / 2) - (-90 + new_step_lat / 2)

In [24]:
lat_axis = xc.create_axis(
    "lat", np.arange(-90 + new_step_lat / 2, 90 + new_step_lat / 2, new_step_lat)
)

# since we have peaked the step such that the latitude range is divded into an integer number of intervals we may center its bounds on -90 / 90.

lon_axis = xc.create_axis("lon", np.arange(0, 360, new_step_lon))

output_grid = xc.create_grid(x=lon_axis, y=lat_axis)

If performing conservative regridding from a high/medium resolution lat/lon grid to a coarse lat/lon target, Regrid2 may provide better results as it assumes grid cells with constant latitudes and longitudes while xESMF assumes the cells are connected by Great Circles : https://xcdat.readthedocs.io/en/latest/generated/xarray.Dataset.regridder.horizontal.html


BASE ARRAY NO VALUE AT THE POLES ??? 

In [26]:
output = ds.regridder.horizontal("sfc_alb", output_grid, tool="regrid2")

fig, axes = plt.subplots(ncols=2, figsize=(16, 4))

ds.sfc_alb.isel(time=0).plot(
    ax=axes[0], vmin=-40, vmax=40, extend="both", cmap="RdBu_r"
)

output.sfc_alb.isel(time=0).plot(
    ax=axes[1], vmin=-40, vmax=40, extend="both", cmap="RdBu_r"
)
axes[1].set_title("Output data")

plt.tight_layout()

In [27]:
ds

In [28]:
max_step_lat

extraire une même variable, regrid toute la variable et combiner sur le nb de modèles.


In [31]:
def regrid_field(ds, field: str):
    """ """

    field_regridded = ds.regridder.horizontal(field, output_grid, tool="regrid2")

    return field_regridded

In [ ]:
keys_aprp = dict_aprp.keys()


fields = [
    "cld",
    "sfc_alb",
    "noncld",
    "noncld_scat",
    "noncld_abs",
    "cld_amt",
    "cld_scat",
    "cld_abs",
]

# initialize

field = "cld"

dict_aprp_regridded_full = {
    key: dict_aprp[key].regridder.horizontal(field, output_grid, tool="regrid2")
    for key in keys_aprp
}


for field in fields[1:]:

    # generate the dictionnary of all models for one field

    dict_aprp_regridded_given_field = {
        key: dict_aprp[key].regridder.horizontal(field, output_grid, tool="regrid2")
        for key in keys_aprp
    }

    # add this one field to the full dictionnary

    dict_aprp_regridded_full = {
        key: add_one_variable_to_dataset(
            variable_name=field,
            var_datarray=dict_aprp_regridded_given_field[key],
            modify_data=True,
            dataset=dict_aprp_regridded_full[key],
        )
        for key in keys_aprp
    }

In [62]:
keys_aprp

In [66]:
keys_aprp = [
    "BCC-ESM1.r1i1p1f1",
    "UKESM1-0-LL.r1i1p1f4",
    "CanESM5.r1i1p2f1",
    "CNRM-CM6-1.r1i1p1f2",
    "CNRM-ESM2-1.r1i1p1f2",
    "ACCESS-ESM1-5.r1i1p1f1",
    "MPI-ESM-1-2-HAM.r1i1p1f1",
    "IPSL-CM6A-LR.r1i1p1f1",
    "IPSL-CM6A-LR-INCA.r1i1p1f1",
    "MIROC6.r1i1p1f1",
    "MRI-ESM2-0.r1i1p1f1",
    "GISS-E2-1-G.r1i1p1f1",
    "CESM2.r1i1p1f1",
    "NorESM2-LM.r1i1p1f1",
    "NorESM2-MM.r1i1p1f1",
    "GFDL-CM4.r1i1p1f1",
]

In [67]:
# needed for the concat function to work

dict_aprp_regridded_full = {
    key: dict_aprp_regridded_full[key]
    .groupby(dict_aprp_regridded_full[key].time.dt.month)
    .mean()
    for key in keys_aprp
}

In [68]:
# let's make the entry average

ensemble_aprp_avg = xr.concat(dict_aprp_regridded_full.values(), "entry").mean(
    dim="entry"
)

## PLOT MAP OF APRP COMPONENTS

In [69]:
output = ensemble_aprp_avg

fields = [
    "cld",
    "sfc_alb",
    "noncld",
    "noncld_scat",
    "noncld_abs",
    "cld_amt",
    "cld_scat",
    "cld_abs",
]

fig = plt.figure(figsize=(12, 12))
plt.suptitle("APRP Components", fontsize=16, x=0, ha="left")
axes = fig.subplots(nrows=4, ncols=2, subplot_kw={"projection": ccrs.Robinson()})
cnt = -1
for row in range(4):
    for col in range(2):
        cnt += 1
        var = fields[cnt]
        avgmap = output.mean("month")
        avg = avgmap.spatial.average(var, axis=["X", "Y"])[var].values
        pl = avgmap[var].plot(
            ax=axes[row, col],
            transform=ccrs.PlateCarree(),
            vmin=-7.5,
            vmax=7.5,
            cmap="RdBu_r",
            add_colorbar=False,
        )
        axes[row, col].set_title(var + " (" + str(np.round(avg, 3)) + ")")
        axes[row, col].coastlines()

plt.tight_layout(w_pad=2.5, h_pad=-2)

fig.colorbar(
    pl, ax=axes.ravel().tolist(), pad=0.02, shrink=0.5, aspect=15, label="W/m$^2$"
)